In [1]:
from utils import *

# load data

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("tmp/")

Extracting tmp/train-images-idx3-ubyte.gz
Extracting tmp/train-labels-idx1-ubyte.gz
Extracting tmp/t10k-images-idx3-ubyte.gz
Extracting tmp/t10k-labels-idx1-ubyte.gz


In [2]:
reset_graph()

n_inputs = 28 * 28
n_hidden1 = 300
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

In [3]:
scale = 0.01 # regularization weight

# use partial to make passing of regularization parameters (and others) easier

from functools import partial

# pass tf.contrib.layers.l1_regularizer to the dense layer
my_dense_layer = partial(
    tf.layers.dense, activation=tf.nn.relu, 
    kernel_regularizer=tf.contrib.layers.l1_regularizer(scale))


with tf.name_scope("dnn"):
    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    logits = my_dense_layer(hidden1, n_outputs, activation=None, name="outputs")

In [4]:

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name="xentropy")
    # calculate conventionaln loss
    base_loss = tf.reduce_mean(xentropy, name="avg_xentropy")
    # get the regularization losses added to the graph by tf.contrib.layers.l1_regularizer
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    # reg_losses is list, hence brackets on base_loss
    # add_n takes everything in list and sums them
    loss = tf.add_n([base_loss] + reg_losses, name="loss")

In [5]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1, name="correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer = tf.summary.FileWriter("to_tensorboard/11_17_l1_reg_tf", tf.get_default_graph())

In [6]:
n_epochs = 20
batch_size = 200

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict= {X:X_batch, y:y_batch})
        accuracy_val = accuracy.eval(feed_dict={X:mnist.validation.images, y:mnist.validation.labels})
        print(epoch, "Validation accuracy:", accuracy_val)
    accuracy_test = accuracy.eval(feed_dict={X:mnist.test.images, y:mnist.test.labels})
    print("Final test accuracy:", accuracy_test)
    
file_writer.close()

0 Validation accuracy: 0.773
1 Validation accuracy: 0.7828
2 Validation accuracy: 0.6972
3 Validation accuracy: 0.6392
4 Validation accuracy: 0.6346
5 Validation accuracy: 0.6404
6 Validation accuracy: 0.6656
7 Validation accuracy: 0.6996
8 Validation accuracy: 0.717
9 Validation accuracy: 0.737
10 Validation accuracy: 0.7596
11 Validation accuracy: 0.766
12 Validation accuracy: 0.7724
13 Validation accuracy: 0.7838
14 Validation accuracy: 0.7874
15 Validation accuracy: 0.7952
16 Validation accuracy: 0.8006
17 Validation accuracy: 0.8024
18 Validation accuracy: 0.8064
19 Validation accuracy: 0.808
Final test accuracy: 0.8086
